In [2]:
from experiments import McGurkExperiment

# Instantiate the list of experiments
experiments = [
    McGurkExperiment("ba", "ga", "da"), # ba (auditory) + ga (visual) = da  (fusioned sound)
    McGurkExperiment("ba", "fa", "va"), # ba (auditory) + fa (visual) = va  (fusioned sound)
    McGurkExperiment("ga", "ba", "bga") # ga (auditory) + ba (visual) = bga (combined sound)
]

In [3]:
from models import McGurkPerceiver

experiment = experiments[0]

# Instantiate a Perceiver model for the given experiment
model = McGurkPerceiver(experiment)

# Train the model
_, _, _, _ = model.train(epochs=50000, learning_rate=0.001)

LogisticRegression: input_dim=512, output_dim=3


Wrongly classified videos:
dataset/train/da/da_jad_2.avi : tensor([4.5973e-01, 5.4027e-01, 9.9527e-40], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_3.avi : tensor([4.7272e-01, 5.2728e-01, 6.6618e-40], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_1.avi : tensor([0.4700, 0.5300, 0.0000], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_4.avi : tensor([4.6805e-01, 5.3195e-01, 1.2564e-38], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_5.avi : tensor([4.8847e-01, 5.1153e-01, 1.6084e-37], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_7.avi : tensor([4.6097e-01, 5.3903e-01, 3.2591e-34], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_6.avi : tensor([4.5400e-01, 5.4600e-01, 2.6518e-35], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_18.avi : tensor([4.7059e-01, 5.2941e-01, 1.4372e-32], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_19.avi : tensor([4.6053e-01, 5.3947e-01, 1.0844e-34], grad_fn=<SelectBackward0>)
dataset/train/da/da_jad_17.avi : tensor([4.3407

In [4]:
# Test the model on McGurk effect videos
predictions = model.test()

# Print the results
for i in range(len(predictions)):
    print(predictions[i])

tensor([6.0985e-05, 9.9994e-01, 9.3509e-22], grad_fn=<SelectBackward0>)
tensor([1.2057e-05, 9.9999e-01, 8.8424e-23], grad_fn=<SelectBackward0>)
tensor([3.3384e-09, 4.9764e-01, 5.0236e-01], grad_fn=<SelectBackward0>)
tensor([7.9002e-05, 9.9992e-01, 5.3723e-21], grad_fn=<SelectBackward0>)
tensor([3.6814e-05, 9.9996e-01, 3.7259e-21], grad_fn=<SelectBackward0>)
tensor([1.6922e-07, 1.0000e+00, 3.2978e-10], grad_fn=<SelectBackward0>)
tensor([2.2273e-06, 1.0000e+00, 7.8300e-09], grad_fn=<SelectBackward0>)
tensor([3.4839e-05, 9.9997e-01, 1.5959e-23], grad_fn=<SelectBackward0>)
tensor([7.8473e-05, 9.9992e-01, 9.2750e-22], grad_fn=<SelectBackward0>)
tensor([5.4733e-05, 9.9995e-01, 2.2401e-23], grad_fn=<SelectBackward0>)
tensor([6.6586e-05, 9.9993e-01, 1.7997e-20], grad_fn=<SelectBackward0>)
tensor([7.8024e-08, 9.9997e-01, 2.8087e-05], grad_fn=<SelectBackward0>)
tensor([9.5818e-07, 1.0000e+00, 7.8007e-09], grad_fn=<SelectBackward0>)
tensor([4.4217e-05, 9.9996e-01, 1.9184e-20], grad_fn=<SelectBack

In [5]:
# Plot average of confidence scores for each sound

import matplotlib.pyplot as plt
import numpy as np

# Get the average confidence scores for each sound
predictions_avg = np.mean(predictions, axis=0)

# Plot the average confidence scores with x axis labels for each sound
plt.bar([experiment.auditory, experiment.visual, experiment.mcgurk], predictions_avg)

TypeError: mean() received an invalid combination of arguments - got (out=NoneType, dtype=NoneType, axis=int, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)
